# FE Homework 8
**FBA Quant 이의형**

---

## Problem 1.

Suppose you are a portfolio manager and you are going to use derivatives to construct certain portfolio.  
The current stock price for AAPL is \\$171.01 per share, at 10 am, March 8, 2019.  
You want to sell 1 unit of European Call on AAPL, with strike of \\$180 and maturity of 1 year.  
Suppose the annualized interest rate is 3%, the annualized drift for AAPL is 5%, no dividend, and the annualized volatility is 10%.  
In order to hedge the potential risk of your option, you are going to calculate several Greeks based on Black-Merton-Scholes model.

**(a)**  Calculate the Call option price, at 10 am, March 8, 2019.

In [1]:
import numpy as np
from scipy.stats import norm

def black_scholes_equation(S, K, T, r, sigma, type_):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))

    if type_ == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif type_ == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return option_price

In [2]:
cond = dict(S = 171.01,
              K = 180,
              T = 1,
              r = 0.03,
              sigma = 0.10,
              type_ = 'call')

call_price = black_scholes_equation(**cond)

print(f'call option price: {call_price:.4f}')

call option price: 5.2122


**(b)** Calculate the Delta, Gamma, Vega, Theta at 10 am, March 8, 2019. Also, give your thoughts about why Delta is positive or negative.

In [3]:
def get_delta(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    delta = norm.cdf(d1)
    return delta

def get_gamma(S, K, r, T, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    return gamma

def get_vega(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    return vega

def get_theta(S, K, r, T, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))
    theta = - (S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    return theta

In [4]:
cond = dict(S = 171.01,
              K = 180,
              T = 1,
              r = 0.03,
              sigma = 0.10)

print(f'Delta: {get_delta(**cond):.4f}')
print(f'Gamma: {get_gamma(**cond):.4f}')
print(f'Vega: {get_vega(**cond):.4f}')
print(f'Theta: {get_theta(**cond):.4f}')

Delta: 0.4355
Gamma: 0.0230
Vega: 67.3299
Theta: -5.4445


기초자산 가격이 상승할수록 콜옵션의 가치가 높아지기 때문에 Delta는 양수이다.

**(c)** Suppose you are doing a monthly Delta-Hedge, that is, you re-hedge per month. After one month, at 10 am, April 8, 2019, the stock price is \\$180.2, and you want to re-hedge right now. Calculate the additional amount of shares of AAPL you should buy (positive for buying, negative for selling) at 10 am, April 8, 2019, in order to re-hedge. Also, give your thoughts about why you are selling or buying stocks when the stock price goes up from the meaning of Delta Hedging.

In [5]:
cond_init = dict(S = 171.01,
              K = 180,
              T = 1,
              r = 0.03,
              sigma = 0.10)

initial_delta = get_delta(**cond_init)
print(f'Initial Delta: {initial_delta:.4f}')

cond_1m = dict(S = 180.2,
              K = 180,
              T = 11/12,
              r = 0.03,
              sigma = 0.10)

delta_1m = get_delta(**cond_1m)
print(f'Delta after 1 month: {delta_1m:.4f}')
print()
print(f'Additional amount of shares of AAPL: {delta_1m - initial_delta:.4f}')

Initial Delta: 0.4355
Delta after 1 month: 0.6356

Additional amount of shares of AAPL: 0.2001


델타는 기초자산 가격에 따른 옵션 가격의 민감도이므로 콜옵션을 매도한 경우에는 기초자산을 매입하여 헷지한다.  
초기 델타는 0.4355로 1단위당 0.4355만큼 기초자산을 매수하면 포트폴리오의 델타는 0이다.  
1달 뒤 델타는 0.6356으로 1단위당 0.6356만큼 보유해야하므로 0.2001만큼 더 매입해야한다.

**(d)** Calculate the change of your portfolio value (i.e. the net cashflow). Note that your answer should be in $ with new portfolio value minus previous value, in present value at 10 am, April 8, 2019.

In [6]:
option_weight = -1
underlying_weight = 0.4355

cond_init = dict(S = 171.01,
              K = 180,
              T = 1,
              r = 0.03,
              sigma = 0.10,
              type_ = 'call')

init_portfolio_value = black_scholes_equation(**cond_init) * option_weight + cond_init['S'] * underlying_weight

cond_1m = dict(S = 180.2,
              K = 180,
              T = 11/12,
              r = 0.03,
              sigma = 0.10,
              type_ = 'call')

new_portfolio_value = black_scholes_equation(**cond_1m) * option_weight + cond_1m['S'] * underlying_weight

print(f'Change of portfolio value: {new_portfolio_value - init_portfolio_value:.2f} ')

Change of portfolio value: -0.41 


---

## Problem 2.

Assume the same situation as Problem 1.  
Back to at 10 am, March 8, 2019. Suppose you are constructing another portfolio by buying a European Call on AAPL with strike 180 and selling the European Call on AAPL with strike 185.

**(a)** Without volatility skew, that is, volatility is always 10% for different strikes. Calculate your portfolio value V, at 10 am, March 8, 2019.

In [7]:
call_cond = dict(S = 171.01,
              K = 180,
              T = 1,
              r = 0.03,
              sigma = 0.10,
              type_ = 'call')

put_cond = dict(S = 171.01,
              K = 185,
              T = 1,
              r = 0.03,
              sigma = 0.10,
              type_ = 'call')

call_price = black_scholes_equation(**call_cond)
put_price = black_scholes_equation(**put_cond)

print(f'call option price: {call_price:.4f}')
print(f'put option price: {put_price:.4f}')

call option price: 5.2122
put option price: 3.5380


---

## Problem 3.

Solve the corresponding leetcode problem below and register the solution on GitHub.

**Increasing Triplet Subsequence**  

Given an integer array nums, return true if there exists a triple of indices (i, j, k) such that i < j < k and nums[i] < nums[j] < nums[k].  

If no such indices exists, return false.

In [8]:
nums = [2,1,5,0,4,6]
nums

[2, 1, 5, 0, 4, 6]

In [9]:
class Solution1:
    def increasingTriplet(self, nums: list[int]) -> bool:
        if len(set(nums)) < 3:
            return False
        for i in range(len(nums)):
            for j in range(i+1, len(nums)):
                if nums[i] < nums[j]:
                    for k in range(j+1, len(nums)):
                        if nums[j] < nums[k]:
                            return True
        return False

In [10]:
Solution1().increasingTriplet(nums)

True

---

## Problem 4.

Solve the corresponding leetcode problem below and register the solution on GitHub.

**Factorial Trailing Zeroes**

Given an integer n, return the number of trailing zeroes in n!.

Note that n! = n * (n - 1) * (n - 2) * ... * 3 * 2 * 1.

In [11]:
n = 1574
n

1574

In [12]:
#import sys
#sys.set_int_max_str_digits(1000000)

class Solution2:
    
    def factorial(self, n):
        if n == 1 or n == 0:
            return 1
        return n * self.factorial(n-1)

    def trailingZeroes(self, n: int) -> int:
        if n == 0:
            return 0

        num_str = str(self.factorial(n))
        
        cnt = 0
        for s in num_str[::-1]:
            if s == '0':
                cnt += 1
            else:
                break
        return cnt

In [13]:
Solution2().trailingZeroes(n)

390

In [14]:
Solution2().factorial(1574)

3188178904997082852944650454632395301153145851710579075577260277436892987792132454851290897622630757496881047982666970491275190394610323955167148670284138804895063048077754932736591461024136450237039466678836275630270598439363731003794005174085668288980520323496300164846450035056889785488645246665061673734646773878326658717701257166260268833795004483022225370958194194189291862067211191278836319072871566931764904401277867354283903636613871064066005640971339927918002707704296892576634962996562372764100204268471121214163727192698274358253671993215938544513135750089055193551561366203853779337301310167590473723689011522172361202976908563821664883671818897338197173703799121144226188137635105666601031713128424896488552724930921904449307412881218343153894866621375583743315082284313091736089466723752457653203851126299066882920468251697206796168774038328015626940375265179909112604784822142674498847426999578135944041852535391162781497018640580422389008747018864637398019980679979615526992931886298

In [15]:
str(Solution2().factorial(1574))[-391:]

'2000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

---